## DDQN in Hockey Enviroment - Evaluation

Evaluate a trained DDQN against various environments

Base DQN implementation adapted from HW7

In [21]:
import os
import sys
from importlib import reload

import numpy as np
from tqdm.notebook import tqdm

# Adding the parent directory to the path to enable importing
root_dir = os.path.dirname(os.path.abspath("../"))
if root_dir not in sys.path:
    sys.path.append(root_dir)

import DDQN.DDQN as ddqn
from DDQN.DQN import TargetDQNAgent, DoubleDQNAgent
from DDQN.DDQN import DuelingDQNAgent, DoubleDuelingDQNAgent
from DDQN.evaluation import compare_agents, display_stats
import hockey.hockey_env as h_env

reload(h_env)
reload(ddqn)

<module 'DDQN.DDQN' from '/home/kivanc/ders/RL/project/RL-Hockey/DDQN/DDQN.py'>

In [22]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

## Environment & Agent Initialization

In [23]:
env = h_env.HockeyEnv(mode=h_env.Mode.NORMAL)
env.reset()

(array([-3.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  3.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.19867516, -0.67558265,  0.        ,
         0.        ,  0.        ,  0.        ]),
 {'winner': 0,
  'reward_closeness_to_puck': 0.0,
  'reward_touch_puck': 0.0,
  'reward_puck_direction': 0.0})

In [ ]:
STATE_PATH = "../models/hockey_double-ddqn_dev/"

agent_player = DoubleDuelingDQNAgent(
    env.observation_space,
    env.discrete_action_space,
    hidden_sizes=[512],
    hidden_sizes_A=[512, 512],
    hidden_sizes_V=[512, 512],
    use_torch=False
)
agent_player.load_state(STATE_PATH)

agent_opp_weak = h_env.BasicOpponent(weak=True)
agent_opp_strong = h_env.BasicOpponent(weak=False)

## Evaluation

### Winning Rates Against Opponents & Match Statistics

In [26]:
weak_opp_stats = compare_agents(
    agent_player, agent_opp_weak, env, num_matches=1000, tqdm=tqdm
)
strong_opp_stats = compare_agents(
    agent_player, agent_opp_strong, env, num_matches=1000, tqdm=tqdm
)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [28]:
print("#" * 40)
print("Stats Against Weak Opponent:")
display_stats(weak_opp_stats)

print()
print("#" * 40)
print("Stats Against Strong Opponent:")
display_stats(strong_opp_stats)

########################################
Stats Against Weak Opponent:
Player Observation Mean:
  x pos player one: -2.085512425071444
  y pos player one: 2.661450686525624
  angle player one: -0.16862544087174122
  x vel player one: 0.3478612415547019
  y vel player one: 0.787802214157836
  angular vel player one: -0.09975919868787024
  x player two: 1.31187419424935
  y player two: -0.01932853013759281
  angle player two: -0.002281472012421103
  y vel player two: -0.35687117274087693
  y vel player two: -0.006532554771432387
  angular vel player two: -0.0015970454550710352
  x pos puck: -1.0250661493560371
  y pos puck: -0.07513335232927543
  x vel puck: -0.7507639871730074
  y vel puck: -0.045202655977840754
  left player puck keep time: 0.06694466031804948
  right player puck keep time: 0.35197708002273437

Relative Std. Change in Agent Observations:
  x pos player one: -0.5444367802116863
  y pos player one: 0.07498820169727088
  angle player one: 0.6885531072690813
  x vel player 

### Rendered Demonstration Against Strong Opponent

In [31]:
obs_buffer = []
reward_buffer = []
obs, _ = env.reset()
obs_opp = env.obs_agent_two()

done = False
trunc = False
step = 0
while not (done or trunc):
    step += 1
    env.render()

    a1_discr = agent_player.act(obs)
    a1 = env.discrete_to_continous_action(a1_discr)
    a2 = agent_opp_strong.act(obs_opp)

    obs, r, done, trunc, info = env.step(np.hstack([a1, a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)

    obs_opp = env.obs_agent_two()

    if done or trunc:
        print(f"Episode done in {step} steps")
        winner = info["winner"]
        if winner == 0:
            print("Draw")
        else:
            print(f"Winner: " + ("Player" if winner == 1 else "Opponent"))
        break

error: display Surface quit

In [30]:
env.close()